Implement a neural network for m-samples, n-features as we discussed in class (both FP and BP) and for N layers in the hidden layer. Split the data (you can use the log. reg. data or any other one) and train your network with 70% of the data. Use 15% for validation  and test your network with the remaining 15% data. Report the evaluation metrics for varying number of layers in the network. Plot the training loss curves.

In [ ]:
from google.colab import files


uploaded = files.upload()

Saving Logistic_regression_ls.csv to Logistic_regression_ls.csv


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


df = pd.read_csv('Logistic_regression_ls.csv')


#Splitting the data into training and testing sets
n = len(df)
# print(n_sample)
np.random.seed(0)
order = np.random.permutation(n)
x1 = df.x1[order]
x2 = df.x2[order]
y = df.label[order].astype(float)

x1_train = x1[: int(0.7 * n)]
x2_train = x2[: int(0.7 * n)]
y_train = y[: int(0.7 * n)]
x1_val = x1[int(0.7 * n) :int(0.85 * n)]
x2_val = x2[int(0.7 * n) :int(0.85 * n)]
y_val = y[int(0.7 * n) :int(0.85 * n)]
x1_test = x1[int(0.85*n):]
x2_test = x2[int(0.85*n):]
y_test = y[int(0.85*n):]


X_train = np.column_stack((x1_train, x2_train))
X_val = np.column_stack((x1_val, x2_val))
X_test = np.column_stack((x1_test, x2_test))

Y_train = np.array(y_train).reshape(1,len(y_train))
Y_val = np.array(y_val).reshape(1,len(y_val))
Y_test = np.array(y_test).reshape(1,len(y_test))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


m_sample = len(X_train)
n_features = X_train.shape[1]

# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


parameters = {
    "W1": np.random.randn(4, n_features),
    "b1": np.zeros((4, 1)),
    "W2": np.random.randn(4, 4),
    "b2": np.zeros((4, 1)),
    "W3": np.random.randn(1, 4),
    "b3": np.zeros((1, 1))
}
# print(parameters['W1'])
# print(parameters['b1'].shape)



def forward_propagation_new(X,parameters,func = sigmoid):

  W1 = parameters["W1"]
  b1 = parameters["b1"]
  W2 = parameters["W2"]
  b2 = parameters["b2"]
  W3 = parameters["W3"]
  b3 = parameters["b3"]

  # Forward propagation
  Z1 = np.dot(W1, X.T) + b1
  A1 = func(Z1)
  Z2 = np.dot(W2, A1) + b2
  A2 = func(Z2)
  Z3 = np.dot(W3, A2) + b3
  A3 = sigmoid(Z3)


  cache = {
      "Z1": Z1,
      "A1": A1,
      "Z2": Z2,
      "A2": A2,
      "Z3": Z3,
      "A3": A3
  }

  return A3, cache

def backward_propagation_new(X,Y,cache,parameters,iter = 500,learning_rate= 0.01):
  if iter == 0:
        return parameters

  m = X.shape[0]

  # Retrieve parameters and cache
  W1 = parameters["W1"]
  W2 = parameters["W2"]
  W3 = parameters["W3"]
  A1 = cache["A1"]
  A2 = cache["A2"]
  A3 = cache["A3"]


  dZ3 = A3 - Y
  dW3 = np.matmul(dZ3,A2.T)/m
  db3 = np.sum(dZ3, axis=1, keepdims=True) / m
  dA2 = np.matmul(W3.T, dZ3)
  dZ2 = dA2 * A2 * (1 - A2)
  dW2 = np.matmul(dZ2, A1.T) / m
  db2 = np.sum(dZ2, axis=1, keepdims=True) / m
  dA1 = np.matmul(W2.T, dZ2)
  dZ1 = dA1 * A1 * (1 - A1)
  dW1 = np.matmul(dZ1, X) / m
  db1 = np.sum(dZ1, axis=1, keepdims=True) / m



  gradients = {
      "dW1": dW1,
      "db1": db1,
      "dW2": dW2,
      "db2": db2,
      "dW3": dW3,
      "db3": db3
  }

  # Update parameters using gradient descent
  parameters["W1"] = parameters["W1"]- learning_rate * gradients["dW1"]
  parameters["b1"] = parameters["b1"]- learning_rate * gradients["db1"]
  parameters["W2"] = parameters["W2"]- learning_rate * gradients["dW2"]
  parameters["b2"] = parameters["b2"] - learning_rate * gradients["db2"]
  parameters["W3"] = parameters["W3"]- learning_rate * gradients["dW3"]
  parameters["b3"] = parameters["b3"]- learning_rate * gradients["db3"]

  # Forward propagation with updated parameters
  _ , new_cache = forward_propagation_new(X, parameters, sigmoid)

  # Recursive call
  return backward_propagation_new(X, Y, new_cache, parameters, iter-1, learning_rate)



# def loss_msc(a,b):
#   m = len(a)
#   return np.sum(np.square(a-b))/m

#loss clculated using cross entropy function
def compute_loss(a,b):
  J = 0
  for i in range(len(a)):
    J = J + (-b[i]*np.log(sigmoid(a[i]))) - (1-b[i])*np.log(1-sigmoid(a[i]))
    J = J/(len(a))

  return J



# Calculate evaluation metrics
def evaluation(a,b):
  tp = np.sum((a >= 0.5) & (b == 1))
  tn = np.sum((a <= 0.5) & (b == 0))
  fp = np.sum((a >= 0.5) & (b == 0))
  fn = np.sum((a <= 0.5) & (b == 1))

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1_score = 2 * (precision * recall) / (precision + recall)
  accuracy = (tp + tn) / len(b)

  print("Precision:", precision)
  print("Recall:", recall)
  print("F1 Score:", f1_score)
  print("Accuracy:", accuracy)

# # Forward propagation
# A3, cache = forward_propagation_new(X_train, parameters,sigmoid)

# # print(A3)

# # Backward propagation for 1000 iterations with a learning rate of 0.01
# para = backward_propagation_new(X_train, Y_train, cache, parameters)
# # print(para)



# a, cache= forward_propagation_new(X,para,sigmoid)
# # print(a)
# evaluation(a,df.label)

In [ ]:
# Train the model
def train_model(X_train, Y_train, X_validate, Y_validate, parameters, learning_rate, iterations):
    train_losses = []
    validate_losses = []

    for i in range(iterations):
        A, caches = forward_propagation_new(X_train, parameters)
        train_loss = compute_loss(A, Y_train)
        parameters = backward_propagation_new(X_train, Y_train, caches, parameters, iterations, learning_rate)

        # Validation loss
        A_validate, _ = forward_propagation_new(X_validate, parameters)
        validate_loss = compute_loss(A_validate, Y_validate)

        train_losses.append(train_loss)
        validate_losses.append(validate_loss)

    return parameters, train_losses, validate_losses

iterations = 100
learning_rate = 0.01

# Train the model
param, train_losses, validate_losses = train_model(X_train, Y_train, X_val, Y_val, parameters, learning_rate, iterations)


# Test the model
A_test, _ = forward_propagation_new(X_test, param)
evaluation(A_test, Y_test)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 75.0
